In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, precision_score, recall_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import  LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from scipy import stats

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

from xgboost import XGBClassifier

In [ ]:
# Read files from different datasets
PIMA=pd.read_csv('PIMA_diabetes_preprocessed.csv')
Sylhet=pd.read_csv('Sylhet_diabetes_preprocessed.csv')
Mendeley=pd.read_csv('Mendeley_diabetes_preprocessed.csv')
Azure=pd.read_csv('Azure_diabetes_preprocessed.csv')
Fused=pd.read_csv('Fused_diabetes.csv')
BRFSS=pd.read_csv('BRFSS_diabetes_preprocessed.csv')
BRFSS_Demo=pd.read_csv('BRFSS_Demo.csv')

# save all datasets into a dictionary
datasets = {'PIMA': PIMA, 'Sylhet': Sylhet, 'Mendeley': Mendeley, 'Azure': Azure,
             'Fused': Fused, 'BRFSS': BRFSS, 'BRFSS_Demo': BRFSS_Demo}

# Since the gender is the top variance in Mendeley and Azure datasets
# Here the Mendeley and Azure will be separated into two datasets depending on the gender.
# PCA and model fitting will be applied on both separated datasets
Mendeley_Male = Mendeley[Mendeley['Gender'] == 1]
Mendeley_Female = Mendeley[Mendeley['Gender'] == 2]
Azure_Male = Azure[Azure['SEX'] == 1]
Azure_Female = Azure[Azure['SEX'] == 2]

# Save four new datasets into the dictionary
datasets['Mendeley_Male'] = Mendeley_Male
datasets['Mendeley_Female'] = Mendeley_Female
datasets['Azure_Male'] = Azure_Male
datasets['Azure_Female'] = Azure_Female

# Since glucose is the indicator of diabetes, this feature should be dropped for diabetes prediction.
PIMA_NoGlucose = PIMA.drop('Glucose', axis=1)
Mendeley_NoGlucose = Mendeley.drop('HbA1c', axis=1)
Fused_NoGlucose = Fused.drop('Glucose', axis=1)

# Save the datasets without Glucose
datasets['PIMA_NoGlucose'] = PIMA_NoGlucose
datasets['Mendeley_NoGlucose'] = Mendeley_NoGlucose
datasets['Fused_NoGlucose'] = Fused_NoGlucose

print(Mendeley_NoGlucose)

In [ ]:
# function for Principal Components Analysis and visualization
def mypca(n_componets = 5, dataset_name = 'PIMA'):

    # Prepare the training and test sets
    dataset = datasets[dataset_name]
    X = dataset.drop(["Class"], axis = 1)
    scaler = MinMaxScaler() 
    X = scaler.fit_transform(X)

    pca = PCA(n_components=n_componets) # keep the first 5 principal components of the data
    X_pca = pca.fit_transform(X) # Fit PCA on the scaled features

    # visualize the feature space
    plt.plot(np.cumsum(pca.explained_variance_ratio_))
    plt.xlabel('Number of Components')
    plt.ylabel('Cumulative Explained Variance')
    plt.title(f'Explained Variance by Components ({dataset_name})')
    plt.show()

    # visualize the features with loadings
    features = dataset.drop(["Class"], axis=1).columns
    loadings = pd.DataFrame(pca.components_.T, columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5'], index=features)
    pc1_loadings = loadings['PC1'].abs().sort_values(ascending=False) # Find the first principal component

    # Select the top 5 features
    top_features = pc1_loadings.head(5).index.tolist()
    top_loadings = pc1_loadings.head(5).values

    # use a bar plot to visualize the features
    plt.bar(top_features, top_loadings)
    plt.xlabel('Features')
    plt.ylabel('Loading')
    plt.title(f'Top 5 Features for PC1 ({dataset_name})')
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
# Use PCA for feature selection for all four datasets
# mypca(n_componets = 5, dataset_name = 'PIMA')
# mypca(n_componets = 5, dataset_name = 'Sylhet')
# mypca(n_componets = 5, dataset_name = 'Mendeley')
# mypca(n_componets = 5, dataset_name = 'Azure')
# mypca(n_componets=5, dataset_name='Mendeley_Male')
# mypca(n_componets=5, dataset_name='Mendeley_Female')
# mypca(n_componets=5, dataset_name='Azure_Male')
# mypca(n_componets=5, dataset_name='Azure_Female')
mypca(n_componets = 5, dataset_name = 'BRFSS')

In [ ]:
# Switches for different models
RF = False
LR = False
SVM = True
DT = False
XG = False
balancing = False

# Set the dataset
dataset = datasets['BRFSS_Demo']

# Check the dataset
y=dataset['Class']
percent_pos = sum(y)/len(y)
print('Percentage Diabetes cases %.02f %%' %(percent_pos * 100))

# Split the data into features and classes
X = dataset.drop(["Class"], axis = 1)
y = dataset["Class"]

# Normalize the features data
scaler = MinMaxScaler() 
X = scaler.fit_transform(X)

# Create tuned model with parameter grid
if RF:    
    model = RandomForestClassifier(random_state=123, n_jobs=-1) # use all processors for calculation
    # Define the parameter grid
    param_grid = {
    'smote__k_neighbors': [1, 5, 10],  # SMOTE parameters
    'classifier__n_estimators': [100, 200, 300],  # RF parameters
    'classifier__criterion': ['gini', 'entropy', 'log_loss'],
    'classifier__max_features': ['sqrt', 'log2', None]
    }
    
if LR:
    model = LogisticRegression(random_state=123)

if SVM:
    model = SVC(random_state=123, cache_size=2000)
    # Define the parameter grid
    param_grid = {
    'smote__k_neighbors': [1, 5, 10],  # SMOTE parameters
    'classifier__C': [1, 10, 100],  # The strength of the regularization is inversely proportional to C.
    'classifier__kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    # 'classifier__gamma': ['scale', 'auto'], # scale: 1 / (n_features * X.var()); auto: 1 / n_features
    # 'classifier__degree': [1, 3, 5], # relevant for the 'poly' kernel
    'classifier__coef0': [0.0, 0.5, 1.0] # significant for 'poly' and 'sigmoid' kernels
    }

if DT:
    model = DecisionTreeClassifier(random_state=123)
    
if XG:
    model = XGBClassifier(tree_method='hist', device='cpu') # use hist tree and run on all threads of CPU
    # Define the parameter grid
    param_grid = {
        'smote__k_neighbors': [1, 5, 10],  # SMOTE parameters
        'classifier__eta': [0.1, 0.3, 0.9],  # lower eta makes the boosting process more conservative
        'classifier__gamma': [0, 1, 10],  # The larger gamma is, the more conservative the algorithm will be.
        'classifier__max_depth': [0, 6, 12],  # Increasing this value will make the model more complex and more likely to overfit. 0 indicates no limit on depth.
        'classifier__min_child_weight': [0, 1, 2],  # The larger min_child_weight is, the more conservative the algorithm will be.
        'classifier__subsample': [0.1, 0.5, 1], # Setting it to 0.5 means that XGBoost would randomly sample half of the training data prior to growing trees. and this will prevent overfitting.
        # 'classifier__lambda': [0, 1, 10], # Increasing this value will make model more conservative.
        # 'classifier__alpha': [0, 1, 10], # Increasing this value will make model more conservative.
        # 'classifier__refresh_leaf': [0, 1],
        # 'classifier__grow_policy': ['depthwise', 'lossguide'],
        # 'classifier__max_bin': [128, 256, 512]
    }

if balancing:
    smote = SMOTE(random_state=123)

    # Create a pipeline
    pipeline = Pipeline([('smote', smote), ('classifier', model)])

    # Initialize GridSearchCV
    grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=10, scoring='roc_auc') # 10 folds cross-validation, AUC for evaluation
    
    # start the grid search and record the computaion time
    start_time = time.time()
    grid_search.fit(X,y)
    spend_time = time.time()-start_time
    
    #Print the best parameters and AUC score
    print("Best parameters:", grid_search.best_params_)
    print("Best score:", grid_search.best_score_)
    print("Spend time:", spend_time)

else:
    pipeline = Pipeline([('classifier', model)])

    # Remove the first key-value pair to skip the SMOTE
    next(iter(param_grid)) 
    param_grid.pop(next(iter(param_grid)))
    
    # Same as above
    grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=10, scoring='roc_auc')

    start_time = time.time()
    grid_search.fit(X,y)
    spend_time = time.time()-start_time

    # print as above
    print("Best parameters:", grid_search.best_params_)
    print("Best score:", grid_search.best_score_)
    print("Spend time:", spend_time)

In [ ]:
def model_validation(model = 'RF', data = 'PIMA', k_neighbors_smote = 0, 
                     criterion_rf = 'entropy', max_features_rf = 'log2', n_estimators_rf = 600, 
                     C_svm = 0.1, coef0_svm = 0.0, degree_svm = 3, gamma_svm = 'scale', kernel_svm = 'poly', 
                     eta_xg = 0.3, gamma_xg = 10, max_depth_xg = 0, min_child_weight_xg = 2, subsample_xg = 1):
    '''This function reads the dataset and use RF, SVM, or XG Boost model with passed parameters to traning and fit with it. 
    The process will repeat 100 times to get the average AUC and F1 score, as well as the 95% CI'''
    
    # Avoid name duplication
    df = datasets[data]
    
    # Separate features and target variable
    X = df.drop(['Class'], axis=1)
    y = df['Class']

    # save the features
    features = X.columns

    # Normalize the features data
    scaler = MinMaxScaler() 
    X = scaler.fit_transform(X)

    # Initialize lists to store scores
    auc_scores = []
    f1_scores = []

    # Repeat the process 100 times
    for _ in range(100):
        # Split the data randomly each time
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

        if k_neighbors_smote !=0:
            smote = SMOTE(random_state=123, k_neighbors=k_neighbors_smote)
            X_train, y_train = smote.fit_resample(X_train, y_train)
    
        # Initialize the model Classifier
        if model == 'RF':
            clf = RandomForestClassifier(random_state=123, n_jobs=-1, criterion=criterion_rf, max_features=max_features_rf, n_estimators=n_estimators_rf)
        if model == 'SVM':
            clf = SVC(random_state=123, cache_size=2000, C=C_svm, coef0=coef0_svm, degree=degree_svm, gamma=gamma_svm, kernel=kernel_svm)
        if model == 'XG':
            clf = XGBClassifier(tree_method='hist', device='cpu', eta=eta_xg, gamma=gamma_xg, max_depth=max_depth_xg, min_child_weight=min_child_weight_xg, subsample=subsample_xg)
    
        # Fit the model
        clf.fit(X_train, y_train)
    
        # Predict on the test set
        y_pred_proba = clf.predict_proba(X_test)[:, 1]
        y_pred = clf.predict(X_test)
    
        # Calculate scores and append to lists
        auc_scores.append(roc_auc_score(y_test, y_pred_proba))
        f1_scores.append(f1_score(y_test, y_pred))

    # Calculate mean and 95% CI for AUC and F1 score
    auc_mean = np.mean(auc_scores)
    f1_mean = np.mean(f1_scores)
    auc_ci = stats.t.interval(0.95, len(auc_scores)-1, loc=auc_mean, scale=stats.sem(auc_scores))
    f1_ci = stats.t.interval(0.95, len(f1_scores)-1, loc=f1_mean, scale=stats.sem(f1_scores))

    # Identify top 5 most important features
    feature_importances = clf.feature_importances_
    top_features_indices = feature_importances.argsort()[-5:][::-1]  # Get indices of top 5 features
    top_features_names = features[top_features_indices]
    top_features_importances = feature_importances[top_features_indices]

    print(f'AUC Mean: {auc_mean}, 95% CI: {auc_ci}')
    print(f'F1 Mean: {f1_mean}, 95% CI: {f1_ci}')
    print(f'Top 5 Most Important Features in {model} model ({data}):')
    for name, importance in zip(top_features_names, top_features_importances):
        print(f'{name}: {importance}')

    # Create a bar plot for the top 5 features
    plt.figure(figsize=(10, 6))
    plt.bar(top_features_names, top_features_importances, color='skyblue')
    plt.xlabel('Features')
    plt.ylabel('Importance')
    plt.title(f'Top 5 Most Important Features in {model} model ({data})')
    plt.show()

In [ ]:
# model_validation(model = 'RF', data='PIMA', k_neighbors_smote=2, criterion_rf='entropy', max_features_rf='log2', n_estimators_rf=600)
# model_validation(model = 'RF', data='PIMA', k_neighbors_smote=0, criterion_rf='entropy', max_features_rf='log2', n_estimators_rf=500)
# model_validation(model = 'RF', data='Sylhet', k_neighbors_smote=1, criterion_rf='gini', max_features_rf='sqrt', n_estimators_rf=400)
# model_validation(model = 'RF', data='Sylhet', k_neighbors_smote=0, criterion_rf='entropy', max_features_rf='sqrt', n_estimators_rf=500)
# model_validation(model = 'RF', data='Mendeley', k_neighbors_smote=1, criterion_rf='gini', max_features_rf='sqrt', n_estimators_rf=100)
# model_validation(model = 'RF', data='Mendeley', k_neighbors_smote=0, criterion_rf='gini', max_features_rf='sqrt', n_estimators_rf=200)
# model_validation(model = 'RF', data='Mendeley_Male', k_neighbors_smote=1, criterion_rf='entropy', max_features_rf='sqrt', n_estimators_rf=100)
# model_validation(model = 'RF', data='Mendeley_Male', k_neighbors_smote=0, criterion_rf='gini', max_features_rf='sqrt', n_estimators_rf=300)
# model_validation(model = 'RF', data='Mendeley_Female', k_neighbors_smote=1, criterion_rf='gini', max_features_rf='sqrt', n_estimators_rf=100)
# model_validation(model = 'RF', data='Mendeley_Female', k_neighbors_smote=0, criterion_rf='gini', max_features_rf='sqrt', n_estimators_rf=100)
# model_validation(model = 'RF', data='Fused', k_neighbors_smote=10, criterion_rf='entropy', max_features_rf='sqrt', n_estimators_rf=300)
# model_validation(model = 'RF', data='Fused', k_neighbors_smote=0, criterion_rf='entropy', max_features_rf='sqrt', n_estimators_rf=300)
# model_validation(model = 'RF', data='BRFSS_Demo', k_neighbors_smote=5, criterion_rf='entropy', max_features_rf='sqrt', n_estimators_rf=300)

In [ ]:
# model_validation(model = 'XG', data='PIMA', k_neighbors_smote=5, eta_xg=0.3, gamma_xg=10, max_depth_xg=0, min_child_weight_xg=2, subsample_xg=1)
# model_validation(model = 'XG', data='PIMA', k_neighbors_smote=0, eta_xg=0.1, gamma_xg=10, max_depth_xg=0, min_child_weight_xg=0, subsample_xg=1)
# model_validation(model = 'XG', data='PIMA_NoGlucose', k_neighbors_smote=5, eta_xg=0.1, gamma_xg=10, max_depth_xg=0, min_child_weight_xg=2, subsample_xg=0.5)
# model_validation(model = 'XG', data='PIMA_NoGlucose', k_neighbors_smote=0, eta_xg=0.1, gamma_xg=1, max_depth_xg=6, min_child_weight_xg=2, subsample_xg=1)
# model_validation(model = 'XG', data='Sylhet', k_neighbors_smote=10, eta_xg=0.9, gamma_xg=0, max_depth_xg=6, min_child_weight_xg=0, subsample_xg=0.5)
# model_validation(model = 'XG', data='Sylhet', k_neighbors_smote=0, eta_xg=0.3, gamma_xg=0, max_depth_xg=0, min_child_weight_xg=0, subsample_xg=0.5)
# model_validation(model = 'XG', data='Mendeley', k_neighbors_smote=1, eta_xg=0.3, gamma_xg=10, max_depth_xg=0, min_child_weight_xg=2, subsample_xg=1)
# model_validation(model = 'XG', data='Mendeley', k_neighbors_smote=0, eta_xg=0.1, gamma_xg=1, max_depth_xg=0, min_child_weight_xg=0, subsample_xg=0.5)
# model_validation(model = 'XG', data='Mendeley_NoGlucose', k_neighbors_smote=0, eta_xg=0.1, gamma_xg=0, max_depth_xg=0, min_child_weight_xg=2, subsample_xg=0.5)
# model_validation(model = 'XG', data='Mendeley_Male', k_neighbors_smote=5, eta_xg=0.1, gamma_xg=0, max_depth_xg=0, min_child_weight_xg=2, subsample_xg=1)
# model_validation(model = 'XG', data='Mendeley_Male', k_neighbors_smote=0, eta_xg=0.9, gamma_xg=0, max_depth_xg=0, min_child_weight_xg=1, subsample_xg=1)
# model_validation(model = 'XG', data='Mendeley_Female', k_neighbors_smote=1, eta_xg=0.1, gamma_xg=0, max_depth_xg=0, min_child_weight_xg=0, subsample_xg=0.5)
# model_validation(model = 'XG', data='Mendeley_Female', k_neighbors_smote=0, eta_xg=0.1, gamma_xg=0, max_depth_xg=0, min_child_weight_xg=0, subsample_xg=0.5)
# model_validation(model = 'XG', data='Fused_NoGlucose', k_neighbors_smote=1, eta_xg=0.3, gamma_xg=10, max_depth_xg=0, min_child_weight_xg=0, subsample_xg=0.5)
model_validation(model = 'XG', data='Fused_NoGlucose', k_neighbors_smote=0, eta_xg=0.1, gamma_xg=10, max_depth_xg=0, min_child_weight_xg=0, subsample_xg=1)

In [ ]:
# Apply the best model from Fused NoGlucose to the BRFSS dataset
df_train = Fused_NoGlucose

X_train = df_train.drop(['Class'], axis=1)
y_train = df_train['Class']

model = XGBClassifier(tree_method='hist', device='cpu', eta=0.1, gamma=10, max_depth=0, min_child_weight=0, subsample=1)
model.fit(X_train, y_train)

df_test = BRFSS

# Change the name of 'Sex' to 'Gender'
df_test.rename(columns={'Sex': 'Gender'}, inplace=True)

# Set 0 in 'Gender' column as 1, and 1 in 'Gender' column as 2
df_test['Gender'] = df_test['Gender'].replace({0: 2, 1: 1})

# Prepare X_test and y_test
X_test = df_test[['Gender', 'Age', 'BMI']]
y_test = df_test['Class']

# Making predictions
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class
y_pred = model.predict(X_test)  # Get class predictions

# Calculate AUC
auc_score = roc_auc_score(y_test, y_pred_proba)

# Calculate F1 Score
f1_score_result = f1_score(y_test, y_pred)

print(f"AUC: {auc_score}")
print(f"F1 Score: {f1_score_result}")